In [1]:
from torchkf import *
import torch
import numpy as np
import plotly.graph_objs as go
import plotly.express as px

In [9]:
def func(x): 
    has_batch=True
    if len(x.shape) == 1: 
        has_batch=False
        x = x.unsqueeze(0)
    y = torch.bmm(torch.eye(x.shape[1]).expand((x.shape[0], x.shape[1], x.shape[1])), x.unsqueeze(-1)).squeeze(-1)
    if not has_batch:
        y = y[0]
        print('coucou', y.shape)
    return y

# A = UnscentedTransform(func)
A = LinearTransform(torch.randn(4, 4))
H = LinearTransform(torch.randn(2, 4))

In [10]:
state_space = GaussianStateSpaceModel(fwd_transform=A, obs_transform=H, state_dim=4, obs_dim=2)

In [11]:
measurements = torch.Tensor(np.sin(2 * np.pi * np.asarray([[5, 10]]) * np.linspace(0, 10, 1000)[:, None])) + 0.1 * torch.randn(1000,2)

In [12]:
trajectory = state_space.filter(measurements[None, ...], backward_pass=True)

In [ ]:
for i in range(10): 
    trajectory = state_space.filter(measurements[None, ...], backward_pass=True)

In [14]:
plot_traj(trajectory['x_backward'])

In [13]:
state_space.complete_data_likelihood(measurements[None], trajectory)

{'ll': tensor([-5742.6196]),
 'll_x': tensor([-3737.1055]),
 'll_y': tensor([-2005.5140])}

In [6]:
trajectory['x_backward'][:, 1:]

Gaussian(loc: torch.Size([1, 499, 4]), covariance_matrix: torch.Size([1, 499, 4, 4]))

In [28]:
plot_traj(trajectory['y_prior'])

TypeError: '<' not supported between instances of 'torch.Size' and 'int'

In [7]:
means, cov = [t.mean for t in trajectory['x_prior']], [t.covariance_matrix for t in trajectory['x_prior']]
Gaussian(torch.stack(means, dim=1), torch.stack(cov, dim=1)).entropy()

tensor([[ 7.0620,  6.8896,  7.1722,  7.3953,  7.5776,  7.7318,  7.8653,  7.9831,
          8.0884,  8.1838,  8.2708,  8.3508,  8.4249,  8.4939,  8.5584,  8.6191,
          8.6762,  8.7303,  8.7816,  8.8304,  8.8769,  8.9214,  8.9639,  9.0047,
          9.0440,  9.0817,  9.1181,  9.1532,  9.1871,  9.2198,  9.2516,  9.2824,
          9.3122,  9.3412,  9.3694,  9.3968,  9.4234,  9.4494,  9.4747,  9.4994,
          9.5235,  9.5471,  9.5700,  9.5925,  9.6145,  9.6360,  9.6571,  9.6777,
          9.6979,  9.7177,  9.7371,  9.7561,  9.7748,  9.7932,  9.8112,  9.8289,
          9.8463,  9.8634,  9.8802,  9.8967,  9.9130,  9.9290,  9.9447,  9.9602,
          9.9755,  9.9905, 10.0054, 10.0200, 10.0344, 10.0485, 10.0625, 10.0763,
         10.0899, 10.1033, 10.1166, 10.1297, 10.1426, 10.1553, 10.1679, 10.1803,
         10.1926, 10.2047, 10.2167, 10.2285, 10.2402, 10.2518, 10.2632, 10.2745,
         10.2857, 10.2967, 10.3076, 10.3185, 10.3292, 10.3397, 10.3502, 10.3606,
         10.3708, 10.3810, 1

In [14]:
Gaussian(torch.zeros((12, 3,)), torch.eye(3).repeat(12, 1, 1)).batch_shape

torch.Size([12])

In [15]:
torch.linalg.eig(torch.tensor([[[ 4.7787, -1.8454],
         [-1.8454,  3.0973]]]))

torch.return_types.linalg_eig(
eigenvalues=tensor([[5.9659+0.j, 1.9101+0.j]]),
eigenvectors=tensor([[[ 0.8410+0.j,  0.5410+0.j],
         [-0.5410+0.j,  0.8410+0.j]]]))